## Used care listings
We are working with dataset fof used cars from eBay 'Kleinanzeigen', a classfieds section of the German eBay website. The dataset was originally scraped and uploaded to Kaggle, now with a few modifications from the original dataset:

+ now sampled 50,000 data points from the full dataset to ensure the code runs quickly in hosted enviroment
+ Dirtied the dataset a bit to more closely resemable what would have been expected from a scraped dataset

The aim of this proejct is to clearn the data and analyze the included used car listings.


In [1]:
import pandas as pd

dateparser = lambda x:pd.datetime.strptime(x,'%Y-%m-%d %H:%M:%S')
autos = pd.read_csv('/home/tony/Desktop/Excels/autos.csv',encoding='Latin-1',parse_dates=['dateCrawled','dateCreated','lastSeen'],date_parser=dateparser)
autos.head()



,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null datetime64[ns]
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null datetime64[ns]
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50

## Observations
+ data volume of 50000
+ memory usage of 7.6+ MB
+ 20 columns of mostly strings
+ some columns have null values, not more than 20% of the data
+ The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to e more descriptive

In [3]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [4]:
autos = autos.rename(columns={'dateCrawled':'date_crawled', 
                              'yearOfRegistration':'registration_year',
                      'monthOfRegistration':'registration_month',
                      'notRepairedDamage':'unrepaired_damage',
                      'dateCreated':'ad_created',
                      'lastSeen':'last_seen',
                      'postalCode':'postal_code',
                     'nrOfPictures':'number_pictures',
                     'fuelType':'fuel_type',
                     'powerPS':'power_ps',
                     'vehicleType':'vehicle_type',
                     'offerType':'offer_type'})

In [5]:
autos.head()


,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,number_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01,0,39218,2016-04-01 14:38:50


In [6]:
autos.unrepaired_damage



0        nein
1        nein
2        nein
3        nein
4        nein
         ... 
49995    nein
49996    nein
49997    nein
49998    nein
49999    nein
Name: unrepaired_damage, Length: 50000, dtype: object

may be there is a need to thoroughly look through the entry representations of the difference between German to English entries

In [7]:
autos.describe()

,registration_year,power_ps,registration_month,number_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


## Observations
+ let's investigate more into power_ps
+ seems registration year and month might be stored as strings and may need some cleaning
+ We may also have to clean up the trailing longs into floats in each column depending on our usage


+ There are a number of text columns where all (or nearly all) of the values are the same:
+ + seller
+ +  offer_type
+ The num_photos column looks odd, we'll need to investigate this further.

In [8]:
autos = autos.drop(['seller','offer_type','number_pictures'],axis=1)

In [9]:
# import re

# autos.loc[:,['price','odometer']].replace(to_replace='\$([0-9,\.]+).*', value=r'\1',regex=True,inplace=True)


Let's convert the column names from camelcase to snakecase and reword some of the column names based on the data dictionary to e more descriptive

In [10]:
autos['price'] = [i.replace('$','').replace(',','') if '$' in i else i for i in autos['price']]
autos['price'] = autos['price'].astype('int')

In [11]:
autos['odometer'] = [i.replace('km','').replace(',','') if 'km' in i else i for i in autos['odometer']]
autos['odometer'] = autos['odometer'].astype('int')

In [12]:
autos[['price','odometer']].dtypes

price       int64
odometer    int64
dtype: object

In [13]:
autos = autos.rename(columns={'odometer':'odometer_km'})

In [14]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01,39218,2016-04-01 14:38:50


sneak peak of the value counts of those 2 specific columns of odoometer_km and price

In [15]:
print(autos["price"].unique().shape)
print(autos["price"].describe())
autos["price"].value_counts().head(20)

(2357,)
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64

In [16]:
autos.odometer_km.value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

In [17]:
autos.price.value_counts()

0        1421
500       781
1500      734
2500      643
1000      639
         ... 
20790       1
8970        1
846         1
2895        1
33980       1
Name: price, Length: 2357, dtype: int64

In [18]:
autos[['price','odometer_km']].sort_values(by=['price','odometer_km'],ascending=False)

,price,odometer_km
39705,99999999,150000
42221,27322222,40000
27371,12345678,150000
39377,12345678,150000
47598,12345678,150000
...,...,...
47368,0,5000
48193,0,5000
48290,0,5000
48985,0,5000


In [19]:
autos["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [20]:
autos["price"].value_counts().sort_index(ascending=True).head(20)

0     1421
1      156
2        3
3        1
5        2
8        1
9        1
10       7
11       2
12       3
13       2
14       1
15       2
17       3
18       1
20       4
25       5
29       1
30       7
35       1
Name: price, dtype: int64

There are a number of listings with prices below \$30, including about 1,500 at \$0. There are also a small number of listings with very high values, including 14 at around or over $1 million.

Given that eBay is an auction site, there could legitimately be items where the opening bid is \$1. We will keep the \$1 items, but remove anything above \$350,000, since it seems that prices increase steadily to that number and then jump up to less realistic numbers.

In [21]:
# autos = autos[(autos[['price','odometer_km']] > 0).all(axis=1) & (autos[['price','odometer_km']] < 169000).all(axis=1)]

In [22]:
autos = autos[autos['price'].between(1,351000)]
autos.price.describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

# cleaning summary
+ did a sort values of both columns
+ found repeating dubious integer values, went ahead to index in to smallest range out of those values 


## Exploring the date columns
There are a number of columns with date information:

+ date_crawled
+ registration_month
+ registration_year
+ ad_created
+ last_seen
These are a combination of dates that were crawled, and dates with meta-information from the crawler. The non-registration dates are stored as strings.

We'll explore each of these columns to learn more about the listings.

In [23]:
autos = autos.sort_values(by=['date_crawled','ad_created','last_seen'],ascending=True)


sorted autos by date_crawled, ad_created and last_seen from ascending order 

In [24]:
autos['date_crawled'] = pd.to_datetime(autos['date_crawled']).dt.date

In [25]:
autos['last_seen'] = pd.to_datetime(autos['last_seen']).dt.date

changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions 

In [26]:
autos['date_crawled'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

In [27]:
autos['ad_created'].value_counts(normalize=True,dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

In [28]:
autos['last_seen'].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The crawler recorded the date it last saw any listing, which allows us to determine on what day a listing was removed, presumably because the car was sold.

The last three days contain a disproportionate amount of 'last seen' values. Given that these are 6-10x the values from the previous days, it's unlikely that there was a massive spike in sales, and more likely that these values are to do with the crawling period ending and don't indicate car sales.

In [29]:
import datetime as dt
march_date_crawled = autos.loc[(autos['date_crawled'] >= dt.date(2016,3,1)) & (autos['date_crawled'] <= dt.date(2016,4,1)),'date_crawled'].value_counts(normalize=True, dropna=False)

In [30]:
march_date_crawled.sort_index()

2016-03-05    0.029053
2016-03-06    0.016109
2016-03-07    0.041311
2016-03-08    0.038194
2016-03-09    0.037957
2016-03-10    0.036918
2016-03-11    0.037367
2016-03-12    0.042351
2016-03-13    0.017975
2016-03-14    0.041926
2016-03-15    0.039327
2016-03-16    0.033966
2016-03-17    0.036280
2016-03-18    0.014810
2016-03-19    0.039894
2016-03-20    0.043461
2016-03-21    0.042870
2016-03-22    0.037839
2016-03-23    0.036965
2016-03-24    0.033659
2016-03-25    0.036257
2016-03-26    0.036942
2016-03-27    0.035666
2016-03-28    0.039989
2016-03-29    0.039115
2016-03-30    0.038642
2016-03-31    0.036517
2016-04-01    0.038642
Name: date_crawled, dtype: float64

sample percentage distribution of march by date_crawled column

In [31]:
march_last_seen = autos.loc[(autos['last_seen'] > dt.date(2016,3,1)) & (autos['last_seen'] < dt.date(2016,4,1)),'last_seen'].value_counts(normalize=True, dropna=False)

In [32]:
march_last_seen.sort_index()

2016-03-05    0.002525
2016-03-06    0.010196
2016-03-07    0.012721
2016-03-08    0.017479
2016-03-09    0.022626
2016-03-10    0.025151
2016-03-11    0.029180
2016-03-12    0.056079
2016-03-13    0.020975
2016-03-14    0.029715
2016-03-15    0.037434
2016-03-16    0.038794
2016-03-17    0.066226
2016-03-18    0.017333
2016-03-19    0.037337
2016-03-20    0.048699
2016-03-21    0.048650
2016-03-22    0.050398
2016-03-23    0.043698
2016-03-24    0.046611
2016-03-25    0.045300
2016-03-26    0.039619
2016-03-27    0.036900
2016-03-28    0.049184
2016-03-29    0.052680
2016-03-30    0.058409
2016-03-31    0.056079
Name: last_seen, dtype: float64

Sample of distribution percentage of march last_seen logs

In [33]:
march_ad_created = autos.loc[(autos['ad_created'] > dt.date(2016,3,1)) & (autos['ad_created'] < dt.date(2016,4,1)),'ad_created'].value_counts(normalize=True,dropna=False)

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [34]:
march_ad_created.sort_index()

2016-03-02    0.000123
2016-03-03    0.001033
2016-03-04    0.001770
2016-03-05    0.027343
2016-03-06    0.018294
2016-03-07    0.041482
2016-03-08    0.039786
2016-03-09    0.039589
2016-03-10    0.038089
2016-03-11    0.039294
2016-03-12    0.043892
2016-03-13    0.020311
2016-03-14    0.042023
2016-03-15    0.040622
2016-03-16    0.035974
2016-03-17    0.037351
2016-03-18    0.016229
2016-03-19    0.040228
2016-03-20    0.045318
2016-03-21    0.044876
2016-03-22    0.039171
2016-03-23    0.038286
2016-03-24    0.034966
2016-03-25    0.037917
2016-03-26    0.038532
2016-03-27    0.037007
2016-03-28    0.041777
2016-03-29    0.040646
2016-03-30    0.040007
2016-03-31    0.038064
Name: ad_created, dtype: float64

sample of percentage distribution of march ad_created logs

## Summary of exploratory date_crawled,  ad_created, and last_seen logs
+ sorted autos by date_crawled, ad_created and last_seen from ascending order
+ changed last_seen and date_crawled columns to datetime date objects instead of time stamp for percentage distrubutions
+ provided some sample percentages distribution logs for each column

In [35]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

The year that the car was first registered will likely indicate the age of the car. Looking at this column, we note some odd values. The minimum value is 1000, long before cars were invented and the maximum is 9999, many years into the future.

In [36]:
autos.registration_year.sort_values(ascending=False)

33950    9999
8012     9999
38076    9999
49910    9000
25003    8888
         ... 
10556    1800
32585    1800
24511    1111
49283    1001
22316    1000
Name: registration_year, Length: 48565, dtype: int64

In [37]:
autos['registration_year'].between(1900,2018).sum()/autos.shape[0]

0.9995469988674972

 contrary to dataquest solution, we may be using an updated version of the original data set, since our registration_year column has 96% listings between 1900 - 2016

## Dealing with Incorrect Registration Year Data¶
Because a car can't be first registered before the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

One option is to remove the listings with these values. Let's determine what percentage of our data has invalid values in this column:

In [38]:
autos = autos[autos['registration_year'].between(1900,2020)].sort_values(by=['registration_year'],ascending=False)

In [39]:
autos.tail()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
22659,2016-03-14,Opel_Corsa_B,500,test,NaN,1910,NaN,0,corsa,150000,0,NaN,opel,NaN,2016-03-14,52393,2016-04-03
28693,2016-03-22,Renault_Twingo,599,control,kleinwagen,1910,manuell,0,NaN,5000,0,benzin,renault,NaN,2016-03-22,70376,2016-04-06
3679,2016-04-04,Suche_Auto,1,test,NaN,1910,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-04-04,40239,2016-04-04
45157,2016-03-11,Motorhaube,15,control,NaN,1910,NaN,0,NaN,5000,0,NaN,trabant,NaN,2016-03-11,90491,2016-03-25
30781,2016-03-25,Opel_Calibra_V6_DTM_Bausatz_1:24,30,test,NaN,1910,NaN,0,calibra,100000,0,NaN,opel,NaN,2016-03-25,47638,2016-03-26


simply took out the outrageously irrelevant entries which comprised of lesson that 1% of the data


In [40]:
autos.registration_year.value_counts(normalize=True)

2000    0.065012
2005    0.060480
1999    0.059677
2004    0.055680
2003    0.055598
          ...   
1929    0.000021
1939    0.000021
1938    0.000021
1931    0.000021
1952    0.000021
Name: registration_year, Length: 81, dtype: float64

it seems a large percentage of car manufactering occured between 2000 - 2010

In [41]:
# autos.registration_year = pd.to_datetime(autos.registration_year,format="%Y")

In [42]:
# autos.registration_year = pd.DatetimeIndex(autos.registration_year).year

In [43]:
autos.brand.value_counts(normalize=True).head(20)

volkswagen        0.212813
opel              0.108641
bmw               0.108641
mercedes_benz     0.095787
audi              0.085858
ford              0.069667
renault           0.047894
peugeot           0.029457
fiat              0.025996
seat              0.018931
skoda             0.016068
nissan            0.015264
mazda             0.015223
smart             0.014296
citroen           0.014111
toyota            0.012586
hyundai           0.009950
sonstige_autos    0.009641
volvo             0.009043
mini              0.008611
Name: brand, dtype: float64

The Top 5 brands of cars are volkswagen,opel, bmw, mercedez benz, and audi. They comprise of roughly 50 % of the share in entries in the brand column.

In [44]:
dic = {}
for brand in autos.brand.value_counts(normalize=True):
    volkswagen = autos[autos['brand'] == 'volkswagen']
    opel = autos[autos['brand'] == 'opel']
    bmw = autos[autos['brand'] == 'bmw']
    mercedez_benz = autos[autos['brand'] == 'mercedes_benz']
    audi = autos[autos['brand'] == 'audi']
    volks_avg = volkswagen.price.sum() / volkswagen.shape[0]
    dic['volkswagen'] = volks_avg
    opel_avg = opel.price.sum()/opel.shape[0]
    dic['opel'] = opel_avg
    bmw_avg = bmw.price.sum()/bmw.shape[0]
    dic['bmw'] = bmw_avg
    mercedes_avg = mercedez_benz.price.sum()/mercedez_benz.shape[0]
    dic['mercedes_benz'] = mercedes_avg
    audi_avg = audi.price.sum()/audi.shape[0]
    dic['audi'] = audi_avg
    

In [45]:
print(dic)

{'volkswagen': 5333.1962055948115, 'opel': 2941.4664391353813, 'bmw': 8261.382442169132, 'mercedes_benz': 8526.623225806452, 'audi': 9212.9306621881}


+ audi: 9212
+ mercedes_benz: 8526
+ volkswagen: 5333
+ bmw: 8261
+ opel: 2941

In [46]:
mean_mileage = {}
for brand in autos.brand.value_counts(normalize=True):
    volkswagen = autos[autos['brand'] == 'volkswagen']
    opel = autos[autos['brand'] == 'opel']
    bmw = autos[autos['brand'] == 'bmw']
    mercedez_benz = autos[autos['brand'] == 'mercedes_benz']
    audi = autos[autos['brand'] == 'audi']
    mean_mileage['volkswagen'] = volkswagen.odometer_km.sum()/volkswagen.shape[0]
    mean_mileage['opel'] = opel.odometer_km.sum()/opel.shape[0]
    mean_mileage['bmw'] = bmw.odometer_km.sum()/opel.shape[0]
    mean_mileage['mercedes_benz'] = mercedez_benz.odometer_km.sum()/mercedez_benz.shape[0]
    mean_mileage['audi'] = audi.odometer_km.sum()/audi.shape[0]
    
    

In [47]:
print(mean_mileage)

{'volkswagen': 128955.570612719, 'opel': 129452.02882062951, 'bmw': 132682.97307546454, 'mercedes_benz': 130848.3870967742, 'audi': 129492.56238003839}


In [48]:
dic = pd.Series(dic)
mean_mileage = pd.Series(mean_mileage)

In [49]:
dic = pd.DataFrame(dic,columns=['mean_price'])
mean_mileage = pd.DataFrame(mean_mileage,columns=['mean_mileage'])

In [50]:
dic

,mean_price
volkswagen,5333.196206
opel,2941.466439
bmw,8261.382442
mercedes_benz,8526.623226
audi,9212.930662


In [51]:
mean_mileage

,mean_mileage
volkswagen,128955.570613
opel,129452.028821
bmw,132682.973075
mercedes_benz,130848.387097
audi,129492.562380


In [52]:
dic.join(mean_mileage)

,mean_price,mean_mileage
volkswagen,5333.196206,128955.570613
opel,2941.466439,129452.028821
bmw,8261.382442,132682.973075
mercedes_benz,8526.623226,130848.387097
audi,9212.930662,129492.562380


There is no particular relationship between the car brands pricing and its mileage in usage

## Identify categorical data that uses german words
+ translate them, map the values to their english counterparts.


In [53]:
autos.unrepaired_damage.value_counts(normalize=True)

nein    0.881171
ja      0.118829
Name: unrepaired_damage, dtype: float64

In [54]:
autos.unrepaired_damage = ['no' if i == 'nein' else 'yes' if i == 'ja' else i for i in autos.unrepaired_damage]

In [55]:
autos.gearbox.value_counts(normalize=True)

manuell      0.781084
automatik    0.218916
Name: gearbox, dtype: float64

In [56]:
autos.gearbox = ['manual' if i == 'manuell' else 'automatic' if i == 'automatik' else i for i in autos.gearbox]

In [57]:
autos.fuel_type.value_counts(normalize=True)

benzin     0.659413
diesel     0.322307
lpg        0.015001
cng        0.001639
hybrid     0.000831
elektro    0.000427
andere     0.000382
Name: fuel_type, dtype: float64

In [58]:
autos.fuel_type = autos.fuel_type.replace({'benzin':'gasoline','elektro':'electricity','andere':'other'})


In [59]:
autos.head()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
49185,2016-03-28,Alfa_Romeo_156_Sportwagon_1_8_T_Spark,1200,control,NaN,2019,manual,140,156,150000,5,gasoline,alfa_romeo,no,2016-03-28,91058,2016-04-06
5763,2016-03-05,Suche_Auspuff_Anlage_honda_crx_del_sol,100,test,NaN,2019,manual,125,cr_reihe,20000,4,NaN,honda,no,2016-03-05,18106,2016-03-06
47258,2016-03-18,Jaguar_xj40,3500,control,NaN,2018,automatic,222,x_type,150000,4,NaN,jaguar,no,2016-03-18,89269,2016-04-05
38345,2016-03-28,Chevrolet_Aveo_LT___euro_4__1399_cc__TÜV_neu_....,2199,control,NaN,2018,NaN,90,NaN,150000,11,NaN,chevrolet,NaN,2016-03-28,33378,2016-03-31
29864,2016-03-05,VW_Lupo_BJ_1999,700,control,NaN,2018,manual,75,lupo,150000,0,gasoline,volkswagen,yes,2016-03-05,49124,2016-03-11


## Convert dates to tbe uniform numeriuc data
+ e.g. '2016-03-21' becomes 20160321
+ see if there are particular keywords in the name column that you can extract as new columns

In [60]:
def to_integer(dt_time):
    int_array = [10000*i.year + 100*i.month + i.day for i in dt_time]
        
    return int_array

In [61]:
autos.date_crawled = to_integer(autos.date_crawled)


In [62]:
autos.date_crawled.head()

49185    20160328
5763     20160305
47258    20160318
38345    20160328
29864    20160305
Name: date_crawled, dtype: int64

In [63]:
autos.last_seen = to_integer(autos.last_seen)

In [64]:
autos.last_seen.head()

49185    20160406
5763     20160306
47258    20160405
38345    20160331
29864    20160311
Name: last_seen, dtype: int64

In [65]:
autos.ad_created = to_integer(autos.ad_created)

In [66]:
autos.ad_created.head()

49185    20160328
5763     20160305
47258    20160318
38345    20160328
29864    20160305
Name: ad_created, dtype: int64

## lets see if there are keywords in name column that can be extracted as new columns

In [67]:
autos.name.value_counts(normalize=True)

Ford_Fiesta                                            0.001566
Volkswagen_Golf_1.4                                    0.001545
BMW_316i                                               0.001545
BMW_318i                                               0.001483
Volkswagen_Polo                                        0.001483
                                                         ...   
Mercedes_Benz_C180__W202_                              0.000021
Audi_A4_Avant_2.0_TDI_S_line__Navi__18_Zoll__HU_neu    0.000021
Mazda_1.9i_16V__Motor_fuer_MX_5__140_PS                0.000021
Opel_Tigra_1.6i_16V                                    0.000021
2er_pferdeanhaenger_boeckmann                          0.000021
Name: name, Length: 37452, dtype: float64

In [68]:
autos.sort_values(by='name')

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
46266,20160314,!!!!!!!!!!!!!!Renault_Clio_5_Tuerig_Servolenku...,100,control,kleinwagen,1995,manual,60,clio,150000,8,gasoline,renault,no,20160314,53757,20160317
35776,20160305,!!!!!!BMW_325_COUPE_TOP_ZUSTAND_ANSCHAUEN_LOHN...,4549,control,coupe,2000,manual,192,3er,150000,11,gasoline,bmw,no,20160305,13353,20160322
18849,20160329,!!!!!VW_PASSAT_3C_2.0TDI!!!!_SEHR_GEPFLEGT,8399,control,kombi,2006,automatic,140,passat,150000,3,diesel,volkswagen,NaN,20160329,99734,20160406
23991,20160316,!!!!!_Mittwoch_Angebot_!!!!_Nissan_Micra_1.2_A...,5250,test,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160316,48527,20160317
4017,20160314,!!!!!_Montag_Angebot_!!!!_Nissan_Micra_1.2_Ace...,5250,control,kleinwagen,2011,manual,80,micra,100000,3,gasoline,nissan,no,20160314,48527,20160316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,20160309,??_????????_??Top!!!_Peugeot_308_sw_Sport_?...,5999,test,kombi,2008,manual,150,3_reihe,125000,8,gasoline,peugeot,no,20160309,55569,20160407
8199,20160312,°_Mercedes_Benz_190_E_2.0_1.Hd_SEC_Front_ESSD_...,1550,control,limousine,1991,manual,0,c_klasse,150000,9,gasoline,mercedes_benz,NaN,20160312,20251,20160325
48349,20160320,°_°_MERCEDES_C_KLASSE_180_MIT_TÜV_°_°,1150,test,kombi,1997,manual,125,c_klasse,150000,2,gasoline,mercedes_benz,no,20160320,30457,20160407
22770,20160403,»_TWINGO_«,1850,control,kleinwagen,2003,manual,58,twingo,100000,6,gasoline,renault,no,20160403,74321,20160406


sort values of name column via .sort_values

In [69]:
type(autos.name)

pandas.core.series.Series

In [70]:
autos.name = autos.name.str.replace('!|_|»|~|°|%',' ').str.replace('?',' ')

In [71]:
autos = autos.sort_values('brand', ascending=True)

In [72]:
autos.name.nunique()

37451

In [73]:
autos.head(2)

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
49185,20160328,Alfa Romeo 156 Sportwagon 1 8 T Spark,1200,control,NaN,2019,manual,140,156,150000,5,gasoline,alfa_romeo,no,20160328,91058,20160406
9100,20160316,Alfa Romeo 156 2.5 V6 24V Distinctive,5000,test,limousine,2002,manual,192,156,150000,7,gasoline,alfa_romeo,no,20160316,80933,20160317


In [74]:
autos.tail()

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
39887,20160328,XC 90 D5 SPORT DYNAUDIO 7 SITZE 1.HAND S HEFT,8000,test,suv,2008,automatic,185,xc_reihe,90000,1,diesel,volvo,no,20160328,60311,20160328
9114,20160308,Volvo S80 2.4 Automatic,1600,control,limousine,1999,automatic,170,andere,150000,10,gasoline,volvo,no,20160308,48151,20160320
23944,20160404,Volvo 240 GL Family Edition,1800,test,kombi,1993,manual,116,andere,150000,5,gasoline,volvo,no,20160404,80797,20160406
25226,20160306,Volvo XC60 2.4D DRIVe 8 fach bereift/ Scheckheft,21590,control,suv,2010,manual,175,xc_reihe,80000,5,diesel,volvo,no,20160306,24242,20160406
16484,20160330,Volvo V50 2.0D DPF,11000,test,kombi,2006,manual,136,v50,125000,3,diesel,volvo,no,20160330,91732,20160405


In [75]:
autos.loc[autos['brand'] == 'toyota',['brand','name']].loc

In [76]:
autos.iloc[16484,1]

'Unfallwagen zum Herrichten oder Ausschlachten.'

In [77]:
autos['V'] = autos.name.str.extract('(\d+)')

In [78]:
autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen', 'V'],
      dtype='object')

In [79]:
autos

,date_crawled,name,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen,V
49185,20160328,Alfa Romeo 156 Sportwagon 1 8 T Spark,1200,control,NaN,2019,manual,140,156,150000,5,gasoline,alfa_romeo,no,20160328,91058,20160406,156
9100,20160316,Alfa Romeo 156 2.5 V6 24V Distinctive,5000,test,limousine,2002,manual,192,156,150000,7,gasoline,alfa_romeo,no,20160316,80933,20160317,156
30099,20160317,Alfa Romeo 147 1.6 Twin Spark ECO,790,test,limousine,2002,manual,105,147,150000,10,gasoline,alfa_romeo,yes,20160317,79576,20160322,147
42637,20160326,Alfa Romeo Spider/el.Verdeck/Klima/Leder/HU 03...,3450,control,cabrio,1997,manual,150,spider,150000,5,gasoline,alfa_romeo,no,20160326,14789,20160406,03
18892,20160323,Alfa Romeo 147 Tuev bis 02/2018,2390,test,limousine,2006,manual,105,147,150000,3,gasoline,alfa_romeo,no,20160323,49084,20160402,147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39887,20160328,XC 90 D5 SPORT DYNAUDIO 7 SITZE 1.HAND S HEFT,8000,test,suv,2008,automatic,185,xc_reihe,90000,1,diesel,volvo,no,20160328,60311,20160328,90
9114,20160308,Volvo S80 2.4 Automatic,1600,control,limousine,1999,automatic,170,andere,150000,10,gasoline,volvo,no,20160308,48151,20160320,80
23944,20160404,Volvo 240 GL Family Edition,1800,test,kombi,1993,manual,116,andere,150000,5,gasoline,volvo,no,20160404,80797,20160406,240
25226,20160306,Volvo XC60 2.4D DRIVe 8 fach bereift/ Scheckheft,21590,control,suv,2010,manual,175,xc_reihe,80000,5,diesel,volvo,no,20160306,24242,20160406,60


In [80]:
autos.columns

Index(['date_crawled', 'name', 'price', 'abtest', 'vehicle_type',
       'registration_year', 'gearbox', 'power_ps', 'model', 'odometer_km',
       'registration_month', 'fuel_type', 'brand', 'unrepaired_damage',
       'ad_created', 'postal_code', 'last_seen', 'V'],
      dtype='object')

In [81]:

autos = autos[['brand','name','model','V',
               'vehicle_type','fuel_type','gearbox','power_ps','odometer_km','unrepaired_damage',
               'abtest','registration_year','registration_month','date_crawled','ad_created','last_seen','price','postal_code']]

In [82]:
autos.sort_values('brand')

,brand,name,model,V,vehicle_type,fuel_type,gearbox,power_ps,odometer_km,unrepaired_damage,abtest,registration_year,registration_month,date_crawled,ad_created,last_seen,price,postal_code
49185,alfa_romeo,Alfa Romeo 156 Sportwagon 1 8 T Spark,156,156,NaN,gasoline,manual,140,150000,no,control,2019,5,20160328,20160328,20160406,1200,91058
38446,alfa_romeo,Alfo romeo zum verkaufen,147,NaN,limousine,NaN,automatic,0,150000,no,test,2001,1,20160327,20160327,20160329,550,38300
34692,alfa_romeo,Alfa Romeo,NaN,NaN,limousine,gasoline,manual,0,150000,no,test,1998,0,20160315,20160315,20160320,950,79258
48378,alfa_romeo,Alfa Romeo Spider Twin Spark L Cabrio Luxus Au...,spider,NaN,cabrio,gasoline,manual,150,150000,no,test,1998,4,20160402,20160402,20160406,5990,35394
18352,alfa_romeo,Alfa Romeo 147,147,147,kleinwagen,gasoline,manual,0,150000,no,test,2001,0,20160326,20160326,20160406,1000,99734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48541,volvo,Volvo XC90 D5 Aut. Exclusive Topzustand,xc_reihe,90,suv,diesel,automatic,185,90000,no,control,2008,1,20160321,20160321,20160330,16000,22085
20125,volvo,Volvo V50 DPF D2 Kinetic,v50,50,kombi,diesel,manual,114,150000,no,test,2010,6,20160314,20160314,20160405,8700,27576
32835,volvo,Volvo XC60 D5 AWD Aut. Summum,xc_reihe,60,suv,diesel,automatic,205,125000,no,control,2009,9,20160328,20160328,20160401,18990,98554
3192,volvo,Volvo S40 1.8,andere,40,limousine,gasoline,automatic,116,150000,no,control,1999,2,20160321,20160321,20160406,2000,9126


In [83]:
autos.name.str.split(' ')

49185       [Alfa, Romeo, 156, Sportwagon, 1, 8, T, Spark]
9100         [Alfa, Romeo, 156, 2.5, V6, 24V, Distinctive]
30099            [Alfa, Romeo, 147, 1.6, Twin, Spark, ECO]
42637    [Alfa, Romeo, Spider/el.Verdeck/Klima/Leder/HU...
18892               [Alfa, Romeo, 147, Tuev, bis, 02/2018]
                               ...                        
39887    [XC, 90, D5, SPORT, DYNAUDIO, 7, SITZE, 1.HAND...
9114                          [Volvo, S80, 2.4, Automatic]
23944                    [Volvo, 240, GL, Family, Edition]
25226    [Volvo, XC60, 2.4D, DRIVe, , 8, fach, bereift/...
16484                              [Volvo, V50, 2.0D, DPF]
Name: name, Length: 48545, dtype: object

In [84]:
autos.name.str.replace('\d+', '')

49185                 Alfa Romeo  Sportwagon   T Spark
9100                     Alfa Romeo  . V V Distinctive
30099                     Alfa Romeo  . Twin Spark ECO
42637    Alfa Romeo Spider/el.Verdeck/Klima/Leder/HU .
18892                           Alfa Romeo  Tuev bis /
                             ...                      
39887         XC  D SPORT DYNAUDIO  SITZE .HAND S HEFT
9114                               Volvo S . Automatic
23944                         Volvo  GL Family Edition
25226     Volvo XC .D DRIVe   fach bereift/ Scheckheft
16484                                   Volvo V .D DPF
Name: name, Length: 48545, dtype: object

In [85]:
autos.name.str.replace('V','')

49185                Alfa Romeo 156 Sportwagon 1 8 T Spark
9100                   Alfa Romeo 156 2.5 6 24 Distinctive
30099                    Alfa Romeo 147 1.6 Twin Spark ECO
42637    Alfa Romeo Spider/el.erdeck/Klima/Leder/HU 03....
18892                      Alfa Romeo 147 Tuev bis 02/2018
                               ...                        
39887        XC 90 D5 SPORT DYNAUDIO 7 SITZE 1.HAND S HEFT
9114                                olvo S80 2.4 Automatic
23944                           olvo 240 GL Family Edition
25226      olvo XC60 2.4D DRIe  8 fach bereift/ Scheckheft
16484                                     olvo 50 2.0D DPF
Name: name, Length: 48545, dtype: object

In [86]:
autos.name.str.replace('\D+','') 

49185        15618
9100      15625624
30099        14716
42637       032018
18892    147022018
           ...    
39887        90571
9114          8024
23944          240
25226        60248
16484         5020
Name: name, Length: 48545, dtype: object

Might not be a good idea to further explore more keywords in the name column that can be extracted as new columns

## Find most common brand/model combinations


In [87]:
autos.columns

Index(['brand', 'name', 'model', 'V', 'vehicle_type', 'fuel_type', 'gearbox',
       'power_ps', 'odometer_km', 'unrepaired_damage', 'abtest',
       'registration_year', 'registration_month', 'date_crawled', 'ad_created',
       'last_seen', 'price', 'postal_code'],
      dtype='object')

In [88]:
autos[autos['model'] == 'golf']

,brand,name,model,V,vehicle_type,fuel_type,gearbox,power_ps,odometer_km,unrepaired_damage,abtest,registration_year,registration_month,date_crawled,ad_created,last_seen,price,postal_code
24947,volkswagen,Sehr gut erhaltener Golf 3 Servo E Dach,golf,3,kleinwagen,NaN,manual,0,150000,NaN,control,1995,0,20160327,20160327,20160403,650,13583
40065,volkswagen,Volkswagen Golf 1.6 FSI Automatik,golf,1,limousine,gasoline,automatic,116,80000,no,control,2004,9,20160312,20160312,20160329,4999,25524
46937,volkswagen,Volkswagen Golf,golf,NaN,coupe,diesel,manual,90,150000,no,test,2001,10,20160327,20160327,20160329,1400,29683
7866,volkswagen,Volkswagen Golf 3. 1 6 Preis: 350  VB,golf,3,kleinwagen,gasoline,manual,75,150000,NaN,control,1995,0,20160404,20160403,20160406,350,44789
21054,volkswagen,Volkswagen Golf 1.2 TSI BlueMotion Technology ...,golf,1,limousine,gasoline,manual,105,40000,no,control,2010,9,20160324,20160324,20160407,11700,28217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20964,volkswagen,VW Golf III mit fast neuen Winterreifen zu ver...,golf,NaN,kleinwagen,gasoline,manual,60,150000,no,test,1997,4,20160314,20160314,20160406,500,53797
29604,volkswagen,Volkswagen Golf Plus 1.4 TSI,golf,1,bus,gasoline,manual,122,90000,no,test,2008,8,20160316,20160316,20160405,8900,26639
7088,volkswagen,VW Golf III,golf,NaN,kleinwagen,gasoline,manual,60,150000,NaN,test,1997,10,20160313,20160313,20160314,599,57482
17586,volkswagen,VW Golf 5 GTI,golf,5,NaN,gasoline,manual,200,150000,NaN,control,2005,5,20160311,20160311,20160407,5000,75173


In [89]:
autos[autos['model'] == 'andere']

,brand,name,model,V,vehicle_type,fuel_type,gearbox,power_ps,odometer_km,unrepaired_damage,abtest,registration_year,registration_month,date_crawled,ad_created,last_seen,price,postal_code
46627,alfa_romeo,zum Verkauf,andere,NaN,limousine,gasoline,manual,0,150000,no,test,1997,0,20160308,20160308,20160406,500,84478
22209,alfa_romeo,Alfa Romeo 166 2.4 JTD DistinctiveTuev neu Apr...,andere,166,limousine,diesel,manual,140,150000,no,test,2002,2,20160404,20160404,20160406,4000,83737
43327,alfa_romeo,Alfa Romeo GT 3.2 V6 Distinctive,andere,3,coupe,gasoline,manual,241,125000,yes,control,2006,4,20160331,20160331,20160331,3500,81927
9082,alfa_romeo,Alfa Romeo Brera 2.2 JTS 16V Sky View,andere,2,coupe,gasoline,manual,185,50000,no,test,2006,8,20160322,20160322,20160407,14900,65623
38505,alfa_romeo,Alfa Romeo GT 1.9 JTD M Jet DPF Progression,andere,1,coupe,diesel,manual,150,70000,no,control,2010,5,20160326,20160326,20160405,13999,77871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49545,volvo,Volvo seltene Version 244dl,andere,244,limousine,gasoline,automatic,0,150000,NaN,control,1980,1,20160314,20160314,20160406,5000,22177
5149,volvo,Fast Oldtimer Liebhaberfahrzeug  P245 Kombi...,andere,245,kombi,gasoline,automatic,115,150000,no,test,1991,0,20160305,20160305,20160306,2850,73760
19349,volvo,Volvo 66GL blau,andere,66,NaN,gasoline,automatic,57,125000,NaN,test,1980,3,20160308,20160308,20160312,500,36391
9114,volvo,Volvo S80 2.4 Automatic,andere,80,limousine,gasoline,automatic,170,150000,no,control,1999,10,20160308,20160308,20160320,1600,48151


In [90]:
autos.model.value_counts(normalize=True)

golf          0.084566
andere        0.074587
3er           0.058272
polo          0.036621
corsa         0.036447
                ...   
kappa         0.000043
b_max         0.000022
i3            0.000022
rangerover    0.000022
200           0.000022
Name: model, Length: 245, dtype: float64

The most common brand/model combinations is the golf model of the brand Volkswagen. The next most common model/brand combinations is alfa-romeos andere model

## See if average prices follows any patterns based on the milage by splitting odometer_km in to groups

In [91]:
autos.sort_values(['odometer_km','price'])

,brand,name,model,V,vehicle_type,fuel_type,gearbox,power_ps,odometer_km,unrepaired_damage,abtest,registration_year,registration_month,date_crawled,ad_created,last_seen,price,postal_code
41577,alfa_romeo,Alfa Romeo 156 Schlachtfest 2.5 V6 24V 428/B B...,156,156,limousine,gasoline,manual,190,5000,NaN,test,1998,5,20160403,20160403,20160405,1,87743
1987,audi,Audi A4 B5 Ersatzteile,a4,4,limousine,gasoline,NaN,0,5000,NaN,control,1997,0,20160316,20160316,20160407,1,64839
32819,audi,Suche VW / Audi,NaN,NaN,NaN,NaN,NaN,0,5000,NaN,control,2000,0,20160322,20160322,20160405,1,84326
13679,audi,Suche Audi 80 Cabrio,a3,80,NaN,NaN,manual,150,5000,NaN,test,2017,0,20160326,20160326,20160406,1,65558
18569,audi,Suche Leasing Audi A4 Avant A6 oder Q5 SQ5 P...,NaN,4,NaN,NaN,NaN,0,5000,NaN,test,2014,0,20160405,20160405,20160405,1,82166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34375,audi,Audi Audi 20V quattro,andere,20,coupe,gasoline,manual,220,150000,no,control,1990,4,20160326,20160326,20160406,80000,84051
2184,land_rover,Land Rover Vouge,andere,NaN,suv,diesel,automatic,220,150000,no,test,2004,6,20160306,20160306,20160407,99000,13357
17540,porsche,Porsche 930 Turbo classic analytics Gutachten 2+,911,930,coupe,gasoline,manual,300,150000,no,test,1988,6,20160330,20160329,20160406,105000,76829
38814,porsche,Porsche Porsche 964 TURBO S X33 WLS ab Werk 35...,911,964,coupe,gasoline,manual,355,150000,no,test,1992,6,20160322,20160322,20160324,119500,52062


In [92]:
group = autos[['odometer_km','price']].sort_values(['odometer_km','price'])

In [93]:
group

,odometer_km,price
41577,5000,1
1987,5000,1
32819,5000,1
13679,5000,1
18569,5000,1
...,...,...
34375,150000,80000
2184,150000,99000
17540,150000,105000
38814,150000,119500


In [94]:
group.price.rolling(2).apply(lambda x:(x[1]-x[0])/x[0]*100).value_counts()

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


0.000000    40808
1.000000       40
0.909091       38
1.428571       25
2.500000       24
            ...  
0.351254        1
0.317597        1
0.050531        1
0.416667        1
0.145370        1
Name: price, Length: 3317, dtype: int64

after trying to calculate rolling percent changes, there are lots of entries where there is no changes what so ever in price even in the same grouping of odometer_km. let's try to remove that those repeatitive entries and calculate the rolling percent changes again

In [95]:
grouped_drop_duplicate = group.drop_duplicates(subset='price')

In [96]:
grouped_drop_duplicate['PCT_change'] = grouped_drop_duplicate.price.rolling(2).apply(lambda x:round((x[1]-x[0])/x[0]*100,1))

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.
/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
grouped_drop_duplicate.PCT_change.value_counts(normalize=True)

0.1      0.049509
0.3      0.047375
0.2      0.046948
0.4      0.040546
0.5      0.036278
           ...   
28.4     0.000427
64.9     0.000427
27.4     0.000427
16.6     0.000427
100.0    0.000427
Name: PCT_change, Length: 255, dtype: float64

Seems the common trend of odometer_km groups by price is a gradual 10% increase in prices in the same milage range

## Some additional data exploration

In [98]:
milage = autos[['odometer_km','price']]

In [99]:
milage = milage.sort_values('odometer_km')
milage.reset_index().drop(columns='index')

,odometer_km,price
0,5000,4499
1,5000,500
2,5000,699
3,5000,500
4,5000,11000
...,...,...
48540,150000,300
48541,150000,1500
48542,150000,800
48543,150000,400


In [100]:
milage.dtypes

odometer_km    int64
price          int64
dtype: object

In [101]:
milage.odometer_km.unique()

array([  5000,  10000,  20000,  30000,  40000,  50000,  60000,  70000,
        80000,  90000, 100000, 125000, 150000])

In [150]:
grouped = []
for km in milage.odometer_km.unique():
    temp = milage[milage.odometer_km == km].sort_values('price').reset_index().drop(columns=['index','odometer_km'])
    temp.rename(columns={'price':km},inplace=True)
    grouped.append(temp)
    

In [151]:
grouped = pd.concat(grouped,axis=1)

In [152]:
grouped

,5000,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,125000,150000
0,1.0,1.0,2.0,1.0,70.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1
1,1.0,1.0,25.0,20.0,120.0,55.0,14.0,1.0,50.0,100.0,1.0,1.0,1
2,1.0,1.0,30.0,50.0,200.0,60.0,100.0,50.0,80.0,100.0,1.0,1.0,1
3,1.0,10.0,50.0,100.0,300.0,139.0,100.0,100.0,100.0,115.0,1.0,1.0,1
4,1.0,50.0,50.0,156.0,300.0,150.0,117.0,100.0,100.0,150.0,1.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80000
31410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99000
31411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105000
31412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119500


In [154]:
# import pandas_profiling as pp

# pp.ProfileReport(grouped)
grouped.corr()

,5000,10000,20000,30000,40000,50000,60000,70000,80000,90000,100000,125000,150000
5000,1.000000,0.820211,0.797349,0.898615,0.958758,0.520643,0.496994,0.503310,0.478916,0.486047,0.485727,0.426992,0.326931
10000,0.820211,1.000000,0.790588,0.736071,0.736885,0.725206,0.740152,0.768363,0.778026,0.772613,0.740208,0.733513,0.725869
20000,0.797349,0.790588,1.000000,0.838314,0.754161,0.695481,0.689707,0.685704,0.681041,0.686697,0.682634,0.616882,0.574346
30000,0.898615,0.736071,0.838314,1.000000,0.867911,0.795806,0.785704,0.782730,0.771858,0.774200,0.774418,0.717826,0.647111
40000,0.958758,0.736885,0.754161,0.867911,1.000000,0.856761,0.841135,0.845322,0.828297,0.836320,0.834281,0.781637,0.689005
50000,0.520643,0.725206,0.695481,0.795806,0.856761,1.000000,0.858233,0.847200,0.829818,0.824460,0.824234,0.783868,0.717378
60000,0.496994,0.740152,0.689707,0.785704,0.841135,0.858233,1.000000,0.851666,0.807987,0.793491,0.786918,0.749108,0.648841
70000,0.503310,0.768363,0.685704,0.782730,0.845322,0.847200,0.851666,1.000000,0.945344,0.931158,0.924784,0.886834,0.803536
80000,0.478916,0.778026,0.681041,0.771858,0.828297,0.829818,0.807987,0.945344,1.000000,0.910329,0.890761,0.860157,0.774318
90000,0.486047,0.772613,0.686697,0.774200,0.836320,0.824460,0.793491,0.931158,0.910329,1.000000,0.901652,0.858159,0.764852


In [106]:
milage_5000 = autos[autos['odometer_km'] == 5000][['odometer_km','price']].sort_values('price')

In [107]:
milage_5000['PCT_Change'] = milage_5000.price.rolling(2).apply(lambda x:(x[1] - x[0])/x[0] *100)

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [108]:
milage_5000

,odometer_km,price,PCT_Change
41577,5000,1,NaN
27171,5000,1,0.000000
4496,5000,1,0.000000
3679,5000,1,0.000000
36652,5000,1,0.000000
...,...,...,...
40918,5000,198000,2.061856
38299,5000,250000,26.262626
12682,5000,265000,6.000000
35923,5000,295000,11.320755


In [109]:

milage_5000.mean()

odometer_km    5000.000000
price          8621.659781
PCT_Change        1.851476
dtype: float64

In [110]:
milage_40000 = autos[autos['odometer_km'] == 40000][['odometer_km','price']].sort_values('price')

In [111]:
milage_40000['PCT_Change'] = milage_40000.price.rolling(2).apply(lambda x:(x[1] - x[0])/x[0] * 100)

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [112]:
milage_40000

,odometer_km,price,PCT_Change
38669,40000,70,NaN
42869,40000,120,71.428571
17261,40000,200,66.666667
26861,40000,300,50.000000
1124,40000,300,0.000000
...,...,...,...
19216,40000,83000,7.796408
10382,40000,84997,2.406024
38002,40000,85000,0.003530
44943,40000,89000,4.705882


In [113]:

milage_40000.PCT_Change.mean()

1.062303231936887

In [114]:

milage_150000 = autos[autos['odometer_km'] == 150000][['odometer_km','price']].sort_values('price')

In [115]:
milage_150000['PCT_Change']  = milage_150000.price.rolling(2).apply(lambda x:(x[1]-x[0])/ x[0] *100)

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [116]:

milage_150000


,odometer_km,price,PCT_Change
17110,150000,1,NaN
30382,150000,1,0.000000
25281,150000,1,0.000000
11671,150000,1,0.000000
33067,150000,1,0.000000
...,...,...,...
34375,150000,80000,14.297144
2184,150000,99000,23.750000
17540,150000,105000,6.060606
38814,150000,119500,13.809524


In [117]:
milage_150000.PCT_Change.mean()

0.044776549401490076

In [118]:
milage_30_thousands = autos[autos['odometer_km'] == 30000][['odometer_km','price']].sort_values('price')

In [119]:
milage_30_thousands['PCT_Change'] = milage_30_thousands.price.rolling(2).apply(lambda x:(x[1] - x[0])/x[0] * 100)

/home/tony/Desktop/github_repos/Dataquest-modules/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


In [120]:
milage_30_thousands

,odometer_km,price,PCT_Change
16398,30000,1,NaN
37141,30000,20,1900.000000
245,30000,50,150.000000
461,30000,100,100.000000
47102,30000,156,56.000000
...,...,...,...
49391,30000,109999,4.860820
33884,30000,116000,5.455504
43282,30000,119900,3.362069
14268,30000,135000,12.593828


In [121]:
milage_30_thousands.PCT_Change.mean()

3.8161869849673162

In [122]:
autos.groupby('price').groups.keys()

dict_keys([1, 2, 3, 5, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 20, 25, 29, 30, 35, 40, 45, 47, 49, 50, 55, 59, 60, 65, 66, 70, 75, 79, 80, 89, 90, 99, 100, 110, 111, 115, 117, 120, 122, 125, 129, 130, 135, 139, 140, 145, 149, 150, 156, 160, 170, 173, 175, 179, 180, 185, 188, 190, 193, 195, 198, 199, 200, 205, 210, 215, 217, 219, 220, 222, 225, 230, 235, 238, 240, 248, 249, 250, 251, 255, 260, 269, 270, 275, 277, 280, 285, 290, 295, 299, 300, 310, 320, 325, 329, 330, 333, 340, 349, 350, 356, 359, 360, 369, 370, 375, 378, 379, 380, 385, 388, 390, 395, 399, 400, 410, 414, 420, 425, 430, 435, 440, 444, 449, 450, 459, 460, 470, 475, 480, 485, 490, 495, 499, 500, 501, 510, 517, 520, 525, 530, 540, 549, 550, 554, 555, 560, 566, 570, 575, 578, 579, 580, 590, 595, 598, 599, 600, 606, 620, 625, 628, 630, 640, 644, 648, 649, 650, 655, 660, 666, 669, 670, 675, 679, 680, 686, 689, 690, 695, 699, 700, 710, 719, 720, 725, 729, 730, 740, 744, 745, 749, 750, 755, 760, 770, 777, 780, 785, 789, 790, 795, 7

## How much cheaper are cars with damage than their non-damaged counterparts?

In [123]:
autos.columns

Index(['brand', 'name', 'model', 'V', 'vehicle_type', 'fuel_type', 'gearbox',
       'power_ps', 'odometer_km', 'unrepaired_damage', 'abtest',
       'registration_year', 'registration_month', 'date_crawled', 'ad_created',
       'last_seen', 'price', 'postal_code'],
      dtype='object')

In [124]:
autos.sort_values('price')

,brand,name,model,V,vehicle_type,fuel_type,gearbox,power_ps,odometer_km,unrepaired_damage,abtest,registration_year,registration_month,date_crawled,ad_created,last_seen,price,postal_code
38064,bmw,Schlachte e39 touring aspensilber 339,5er,39,kombi,gasoline,manual,193,150000,NaN,control,1996,12,20160329,20160329,20160405,1,26532
23307,opel,Opel vectra c 1.8,vectra,1,kombi,gasoline,manual,122,150000,no,test,2004,4,20160308,20160308,20160308,1,37290
23308,renault,Renault twingo,twingo,NaN,kleinwagen,NaN,manual,58,125000,NaN,control,1998,0,20160321,20160321,20160322,1,59519
15731,opel,Opel Zafira 1 9 CDTI schoene 7 Sitze fuer dein...,zafira,1,bus,diesel,manual,150,150000,NaN,control,2009,4,20160402,20160401,20160407,1,56075
8322,fiat,Fiat Punto 1.2 SCHLACHTFEST,punto,1,kleinwagen,gasoline,manual,60,150000,yes,test,2000,3,20160311,20160311,20160326,1,25813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12682,porsche,Porsche GT3 RS PCCB Lift grosser Exklusiv ...,911,3,coupe,gasoline,automatic,500,5000,no,control,2016,3,20160328,20160328,20160405,265000,70193
35923,porsche,Porsche 911 Targa Exclusive Edition 1 von 15 ...,911,911,cabrio,gasoline,automatic,400,5000,no,test,2015,6,20160403,20160403,20160403,295000,74078
34723,porsche,Porsche Porsche 911/930 Turbo 3.0 deutsche Au...,911,911,coupe,gasoline,manual,260,100000,no,test,1977,7,20160323,20160323,20160406,299000,61462
14715,sonstige_autos,Rolls Royce Phantom Drophead Coupe,NaN,NaN,cabrio,gasoline,automatic,460,20000,no,control,2012,8,20160330,20160330,20160407,345000,73525


In [125]:
damages = autos[['unrepaired_damage','price']]

In [126]:
damages.groupby('unrepaired_damage')['price'].mean()

unrepaired_damage
no     7086.102355
yes    2221.887609
Name: price, dtype: float64

grouping by unrepaired_damage to find the mean price between cars having no unrepaired_damage and repaired damange, the price difference about $4500 where repaired cars are values less